In [1]:
import warnings
warnings.filterwarnings("ignore")

import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import os
import random

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import timm

import albumentations as A
from albumentations.pytorch import ToTensorV2

In [ ]:

PATH = "./train"
TEST_PATH = "./test"

IMG_SIZE = 224  

LABELS = ['REAL', 'FAKE']

train_dataset = datasets.ImageFolder(PATH, transform=None)
test_dataset = datasets.ImageFolder(TEST_PATH, transform=None)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

class_names = train_dataset.classes
print(f"Classes: {class_names}")

Classes: ['FAKE', 'REAL']


In [3]:
train_transforms = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

test_transforms = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset

class AlbumentationsDataset(Dataset):
    def __init__(self, imagefolder_dataset, transform=None):
        self.imagefolder_dataset = imagefolder_dataset
        self.transform = transform

    def __len__(self):
        return len(self.imagefolder_dataset)

    def __getitem__(self, idx):
        # Get PIL image and label
        image, label = self.imagefolder_dataset[idx]

        # Convert PIL -> numpy (H,W,C)
        image = np.array(image)

        # Apply Albumentations transform
        if self.transform:
            transformed = self.transform(image=image)
            image = transformed["image"]  # Should be Tensor [C,H,W]
        else:
            # fallback to torch tensor
            image = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0

        #  Final safeguard: ensure channel-first 3-channel
        if isinstance(image, np.ndarray):
            image = torch.from_numpy(image)
        if image.ndim == 3 and image.shape[0] != 3:
            image = image.permute(2, 0, 1)
        if image.shape[0] != 3:
            raise ValueError(f"Unexpected channel shape: {image.shape}")

        return image, label


In [5]:
train_aug_dataset = AlbumentationsDataset(train_dataset, transform=train_transforms)
test_aug_dataset = AlbumentationsDataset(test_dataset, transform=test_transforms)

train_loader = DataLoader(train_aug_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_aug_dataset, batch_size=32, shuffle=False)

In [6]:
vgg16_model = models.vgg16(weights='IMAGENET1K_V1')
vgg16_model.classifier[6] = nn.Linear(4096, len(LABELS))
vgg16_model = vgg16_model.to("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
import timm
import torch.nn as nn
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# 1. Load pretrained model
efficientnet_model = timm.create_model("efficientnet_b3", pretrained=True)

# 2. Freeze all feature extractor layers
for param in efficientnet_model.parameters():
    param.requires_grad = False

# 3. Replace classifier head (for binary classification)
in_features = efficientnet_model.classifier.in_features
efficientnet_model.classifier = nn.Sequential(
    nn.Linear(in_features, 256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 1),
    nn.Sigmoid()  # for binary output
)

efficientnet_model = efficientnet_model.to(device)


In [13]:
from tqdm import tqdm
import time

def train_model(model, optimizer, criterion, train_loader, test_loader, num_epochs=5):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    best_val_acc = 0
    start_time = time.time()

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        epoch_start = time.time()

        model.train()
        running_loss = 0
        correct = 0
        total = 0

        train_bar = tqdm(train_loader, desc="Training", leave=False)
        for images, labels in train_bar:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_acc = correct / total

        model.eval()
        correct = 0
        total = 0

        val_bar = tqdm(test_loader, desc="Validation", leave=False)
        with torch.no_grad():
            for images, labels in val_bar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = correct / total
        elapsed = time.time() - epoch_start
        total_elapsed = time.time() - start_time
        est_remaining = (total_elapsed / (epoch + 1)) * (num_epochs - epoch - 1)

        print(f"Epoch {epoch+1}: Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f} | Time: {elapsed:.1f}s | ETA: {est_remaining/60:.1f} min")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), f"best_{model.__class__.__name__}.pth")

    total_time = time.time() - start_time
    print(f"\nTraining complete. Best Val Acc: {best_val_acc:.4f} | Total Time: {total_time/60:.1f} min")

In [ ]:
efficientnet_optimizer = optim.AdamW(efficientnet_model.parameters(), lr=1e-4, weight_decay=0.01)
efficientnet_criterion = nn.BCELoss()
images, labels = next(iter(train_loader))
print("Batch shape:", images.shape)

start_time = time.time()
train_model(
    model=efficientnet_model,
    optimizer=efficientnet_optimizer,
    criterion=efficientnet_criterion,
    train_loader=train_loader,
    test_loader=test_loader,
    num_epochs=5
)
print("EfficientNet-B3 training completed in {:.2f} minutes.".format((time.time() - start_time) / 60))

Batch shape: torch.Size([32, 3, 224, 224])
Total parameters: 11,090,473
Trainable parameters: 394,241
Percentage trainable: 3.55%

Epoch 1/5


Training:   0%|          | 0/3125 [00:00<?, ?it/s]

Validation:   0%|          | 0/625 [00:00<?, ?it/s]

Epoch 1: Train Acc=0.8448, Val Acc=0.8843 | Time: 2052.6s | ETA: 136.8 min

Epoch 2/5


Training:   0%|          | 0/3125 [00:00<?, ?it/s]

Validation:   0%|          | 0/625 [00:00<?, ?it/s]

Epoch 2: Train Acc=0.8698, Val Acc=0.8919 | Time: 499.1s | ETA: 63.8 min

Epoch 3/5


Training:   0%|          | 0/3125 [00:00<?, ?it/s]

In [14]:
efficientnet_model_2 = timm.create_model("efficientnet_b3", pretrained=True, num_classes=len(LABELS))
efficientnet_model_2 = efficientnet_model_2.to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
efficientnet_optimizer = optim.AdamW(efficientnet_model_2.parameters(), lr=1e-4, weight_decay=0.01)
efficientnet_criterion = nn.CrossEntropyLoss()

start_time = time.time()
train_model(
    model=efficientnet_model_2,
    optimizer=efficientnet_optimizer,
    criterion=efficientnet_criterion,
    train_loader=train_loader,
    test_loader=test_loader,
    num_epochs=5
)
print("EfficientNet-B3 training completed in {:.2f} minutes.".format((time.time() - start_time) / 60))


Epoch 1/5


Training:   0%|          | 0/3125 [00:00<?, ?it/s]

Epoch 1: Train Acc=0.9301, Val Acc=0.9742 | Time: 5675.6s | ETA: 378.4 min

Epoch 2/5


Epoch 2: Train Acc=0.9727, Val Acc=0.9801 | Time: 5471.2s | ETA: 278.7 min

Epoch 3/5


Epoch 3: Train Acc=0.9837, Val Acc=0.9828 | Time: 5421.5s | ETA: 184.1 min

Epoch 4/5


Epoch 4: Train Acc=0.9881, Val Acc=0.9860 | Time: 5701.4s | ETA: 92.8 min

Epoch 5/5


Epoch 5: Train Acc=0.9913, Val Acc=0.9858 | Time: 6113.6s | ETA: 0.0 min

Training complete. Best Val Acc: 0.9860 | Total Time: 473.1 min
✅ EfficientNet-B3 training completed in 473.08 minutes.


In [14]:
import torch
import gc

# Clear all GPU caches
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

# Force Python garbage collection
gc.collect()

# If you have any model/optimizer loaded, delete them first
# del model, optimizer, etc.


3711

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = models.vgg16(pretrained=False).eval().to(device)

x = torch.randn(1, 3, 224, 224).to(device)

params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {params:,}")

for _ in range(5):
    with torch.no_grad():
        _ = model(x)

torch.cuda.synchronize()
start = time.time()

with torch.no_grad():
    _ = model(x)

torch.cuda.synchronize()
end = time.time()

print(f"Inference time: {(end - start)*1000:.2f} ms")


Using device: cuda
Total parameters: 138,357,544
Inference time: 9.41 ms
